In [ ]:
!pip install prince

In [108]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# 차원축소
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
import prince


In [109]:
data = pd.read_csv('./data_to_model/세대원_상가_제로_유치원_환경_도시농업_초등학교_사업체_채식_인구수_연령별_카페_유동인구_매출_sum.csv', index_col=0)
data.fillna(0, inplace=True)
data.head(3)

,2023년09월_전체세대,2023년09월_1인세대,2023년09월_2인세대,2023년09월_3인세대,2023년09월_4인세대,2023년09월_5인세대,2023년9월_6인가구이상,채소가게수,제로웨이스트 가게수,유치원수,...,2_40대,2_50대,2_60대,2_70대,2_80대,2_90대,2_기타,카페수,승하차인원_버스_지하철,1점포당_1년_매출액
가락1동,9836,1633,2492,2706,2438,449,118,10.0,0.0,1.0,...,4892,3948,3832,1672,478,94,4,43.0,184724.0,1.175032e+08
가락2동,12375,3568,2746,2825,2641,453,142,3.0,0.0,4.0,...,5304,5224,4686,1834,840,182,8,74.0,190052.0,3.372717e+07
가락본동,11667,4854,2640,2123,1567,385,98,9.0,2.0,2.0,...,3520,4500,4002,1718,768,132,4,99.0,273627.0,3.056880e+07


In [110]:
df = data.copy()
df.columns

Index(['2023년09월_전체세대', '2023년09월_1인세대', '2023년09월_2인세대', '2023년09월_3인세대',
       '2023년09월_4인세대', '2023년09월_5인세대', '2023년9월_6인가구이상', '채소가게수',
       '제로웨이스트 가게수', '유치원수', '거주인구대비취약자수', '거주인구대비취약자백분위수', '미세먼지지수',
       '미세먼지백분위', '행정구역면적', '공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)', '초등학교수',
       '사업체수 (개)', '종사자수 (명)', '채식식당수', '인구수', '1_0대', '1_10대', '1_20대',
       '1_30대', '1_40대', '1_50대', '1_60대', '1_70대', '1_80대', '1_90대', '1_기타',
       '2_0대', '2_10대', '2_20대', '2_30대', '2_40대', '2_50대', '2_60대', '2_70대',
       '2_80대', '2_90대', '2_기타', '카페수', '승하차인원_버스_지하철', '1점포당_1년_매출액'],
      dtype='object')

In [111]:
# 인구 범위 축소하기
df['남자_10대이하'] = df['1_0대'] + df['1_10대']
df['남자_20_30대'] = df['1_20대'] + df['1_30대']
df['남자_40_60대'] = df['1_40대'] + df['1_50대'] + df['1_60대']
df['남자_70대이상'] = df['1_70대'] + df['1_80대'] + df['1_90대'] + df['1_기타']

df['여자_10대이하'] = df['2_0대'] + df['2_10대']
df['여자_20_30대'] = df['2_20대'] + df['2_30대']
df['여자_40_60대'] = df['2_40대'] + df['2_50대'] + df['2_60대']
df['여자_70대이상'] = df['2_70대'] + df['2_80대'] + df['2_90대'] + df['2_기타']

# 유치원, 초등학교 수 합치기
df['유치원_초등학교'] = df['유치원수'] + df['초등학교수']

In [112]:
df.columns

Index(['2023년09월_전체세대', '2023년09월_1인세대', '2023년09월_2인세대', '2023년09월_3인세대',
       '2023년09월_4인세대', '2023년09월_5인세대', '2023년9월_6인가구이상', '채소가게수',
       '제로웨이스트 가게수', '유치원수', '거주인구대비취약자수', '거주인구대비취약자백분위수', '미세먼지지수',
       '미세먼지백분위', '행정구역면적', '공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)', '초등학교수',
       '사업체수 (개)', '종사자수 (명)', '채식식당수', '인구수', '1_0대', '1_10대', '1_20대',
       '1_30대', '1_40대', '1_50대', '1_60대', '1_70대', '1_80대', '1_90대', '1_기타',
       '2_0대', '2_10대', '2_20대', '2_30대', '2_40대', '2_50대', '2_60대', '2_70대',
       '2_80대', '2_90대', '2_기타', '카페수', '승하차인원_버스_지하철', '1점포당_1년_매출액',
       '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상', '유치원_초등학교'],
      dtype='object')

In [113]:
population_feature = ['2023년09월_1인세대', '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상']

infra_feature = ['공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)']
demand_feature = ['채소가게수', '카페수', '채식식당수', '초등학교수', '유치원수']
environ_featrue = ['미세먼지지수', '미세먼지백분위', '거주인구대비취약자수']
market_feature = ['1점포당_1년_매출액', '승하차인원_버스_지하철', '종사자수 (명)', '사업체수 (개)']

In [75]:
# df = df[population_feature + infra_feature + demand_feature + environ_featrue + market_feature]

In [114]:
scaled_data2 = StandardScaler().fit_transform(np.log1p(data))
data_scale_log2=pd.DataFrame(data=scaled_data2, columns=data.columns)

In [115]:
scaled_data = StandardScaler().fit_transform(np.log1p(df))
data_scale_log=pd.DataFrame(data=scaled_data, columns=df.columns)
data_scale_log

,2023년09월_전체세대,2023년09월_1인세대,2023년09월_2인세대,2023년09월_3인세대,2023년09월_4인세대,2023년09월_5인세대,2023년9월_6인가구이상,채소가게수,제로웨이스트 가게수,유치원수,...,1점포당_1년_매출액,남자_10대이하,남자_20_30대,남자_40_60대,남자_70대이상,여자_10대이하,여자_20_30대,여자_40_60대,여자_70대이상,유치원_초등학교
0,0.082020,-1.232252,0.298329,0.868037,1.054325,0.838395,0.760275,1.691391,-0.602463,-0.648543,...,1.762880,1.244875,0.477965,0.643055,0.257813,1.232333,0.592844,0.683955,-0.122153,-0.018742
1,0.481592,-0.104676,0.458849,0.932058,1.161431,0.850548,1.040792,0.284733,-0.602463,1.121062,...,0.077817,1.196505,0.743152,0.933382,0.512298,1.129477,0.760896,0.960985,0.257900,1.034959
2,0.379076,0.339458,0.393743,0.507109,0.462467,0.627721,0.479326,1.558860,1.561220,0.134520,...,-0.054924,0.417238,0.693772,0.641965,0.340402,0.439900,0.612464,0.604165,0.119354,0.401415
3,-0.750427,0.215361,-1.232976,-1.958958,-2.224547,-2.630133,-3.565654,0.848543,-0.602463,-1.987197,...,-1.760553,-2.918667,-1.585274,-2.404177,-1.323867,-3.141218,-2.004816,-2.817779,-1.522056,-1.323869
4,1.139489,1.876298,0.132325,-0.618365,-0.893192,-0.920683,-0.943036,1.248573,-0.602463,-0.648543,...,1.089504,-0.733127,1.503474,0.313347,-0.142810,-0.709616,1.289160,-0.243285,-0.422832,-1.323869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,-1.194064,-1.128324,-0.932169,-0.821133,-0.604630,-0.688738,-0.186748,-0.679107,-0.602463,-1.987197,...,0.815518,-0.488349,-0.598500,-0.897183,-1.227573,-0.528896,-0.478198,-0.800295,-0.976641,-1.323869
422,-0.218677,0.147710,-0.237775,-0.362553,-0.513086,-0.175921,-0.028264,-1.642947,0.762669,-0.648543,...,0.050870,-0.165858,0.011571,-0.168064,-0.267126,-0.258045,-0.111101,-0.102322,0.006459,-0.018742
423,-0.328915,0.277813,-0.742428,-0.788983,-0.633360,-0.547766,-0.658144,-1.642947,-0.602463,-0.648543,...,0.042364,-0.500225,0.053614,-0.687745,-0.600937,-0.411399,0.264334,-0.708255,-0.536773,-1.323869
424,0.536539,0.840924,0.276282,0.180988,0.248957,0.387268,0.134756,-0.679107,-0.602463,0.690111,...,-0.631576,0.366951,0.905033,0.336318,0.416288,0.435516,0.683939,0.306523,0.414241,0.744708


#### 인구특성



In [116]:
population_feature = ['2023년09월_1인세대', '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상']

pca = PCA(n_components=0.95)
population_scale_data = data_scale_log[population_feature]
pca.fit(population_scale_data)
pca_population = pd.DataFrame((pca.transform(population_scale_data)))

num_of_principal = pca_population.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 3


In [117]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=population_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.8243068415827797
제 2주성분 : 0.11777893165341181
제 3주성분 : 0.037538521129754754


,0,1,2
2023년09월_1인세대,-0.248212,0.691402,0.109357
남자_10대이하,-0.323765,-0.422133,0.231143
남자_20_30대,-0.344253,0.268372,0.266949
남자_40_60대,-0.358644,-0.102116,-0.036978
남자_70대이상,-0.345005,0.038929,-0.550983
여자_10대이하,-0.323410,-0.419514,0.258925
여자_20_30대,-0.342574,0.232359,0.395523
여자_40_60대,-0.357236,-0.168928,-0.031918
여자_70대이상,-0.343270,0.035784,-0.577844


In [118]:
population_columns = ['인구가 비교적 적은 지역', '1인 세대 및 20~30대 남녀가 많은 지역', '미취학 및 초등학생, 젊은 세대가 많은 지역']

In [119]:
population_feature2 = ['2023년09월_전체세대', '2023년09월_1인세대', '2023년09월_2인세대', '2023년09월_3인세대',
       '2023년09월_4인세대', '2023년09월_5인세대', '2023년9월_6인가구이상', '인구수', '1_0대', '1_10대', '1_20대', '1_30대', '1_40대', '1_50대', '1_60대',
       '1_70대', '1_80대', '1_90대', '1_기타', '2_0대', '2_10대', '2_20대', '2_30대',
       '2_40대', '2_50대', '2_60대', '2_70대', '2_80대', '2_90대', '2_기타']

pca = PCA(n_components=0.9)
population_scale_data2 = data_scale_log2[population_feature2]
pca.fit(population_scale_data2)
pca_population2 = pd.DataFrame((pca.transform(population_scale_data2)))

num_of_principal = pca_population2.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 3


In [120]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=population_scale_data2.columns).T

주성분의 설명력
제 1주성분 : 0.7595317466640681
제 2주성분 : 0.07778940163742226
제 3주성분 : 0.06628451523468712


,0,1,2
2023년09월_전체세대,-0.188225,-0.085634,0.190318
2023년09월_1인세대,-0.130043,-0.364080,0.289448
2023년09월_2인세대,-0.194396,-0.010412,0.168612
2023년09월_3인세대,-0.193151,0.206735,0.046000
2023년09월_4인세대,-0.180555,0.308667,-0.018286
2023년09월_5인세대,-0.182603,0.286665,-0.001836
2023년9월_6인가구이상,-0.172022,0.289106,0.007145
인구수,-0.207371,-0.044955,-0.007855
1_0대,-0.189573,0.201952,-0.096559
1_10대,-0.187043,0.230800,-0.131348


### 환경 관련 인프라 특성

In [121]:
infra_feature = ['공원율', '도시농업(프로그램수)', '도시농업(시설수)']

pca = PCA(n_components=0.9)
infra_scale_data = data_scale_log[infra_feature]
pca.fit(infra_scale_data)
pca_infra = pd.DataFrame((pca.transform(infra_scale_data)))

num_of_principal = pca_infra.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 2


In [122]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=infra_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.6598873059040783
제 2주성분 : 0.32879078536359385


,0,1
공원율,0.117606,0.993002
도시농업(프로그램수),0.701757,-0.090786
도시농업(시설수),0.702642,-0.075534


In [123]:
infra_column = ['도시농업 체험 프로그램이 많은 지역', '도시 농업 체험 프로그램이 적은 지역']

### 예상 수요 특성

In [127]:
demand_feature = ['채소가게수', '카페수', '채식식당수', '초등학교수', '유치원수']
# demand_feature = ['채소가게수', '카페수', '채식식당수', '유치원_초등학교']

pca = PCA(n_components=0.9)
demand_scale_data = data_scale_log[demand_feature]
pca.fit(demand_scale_data)
pca_demand = pd.DataFrame((pca.transform(demand_scale_data)))

num_of_principal = pca_demand.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 4


In [128]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=demand_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.36276318573145905
제 2주성분 : 0.30545785029050854
제 3주성분 : 0.1892536332273014
제 4주성분 : 0.09390414272839434


,0,1,2,3
채소가게수,-0.306171,-0.009671,0.928065,0.131210
카페수,-0.689894,0.043408,-0.088282,-0.078886
채식식당수,-0.655775,-0.033026,-0.337349,-0.001372
초등학교수,-0.007096,0.705509,-0.091342,0.702558
유치원수,0.014627,0.706533,0.093567,-0.694961


In [129]:
demand_column = ['예상 수요가 낮은 지역', '유치원이 많은 지역', '납품할 업체가 많은 지역', '초등학교가 많은 지역']

### 환경 오염 특성

In [130]:
environ_feature = ['미세먼지지수', '미세먼지백분위', '거주인구대비취약자수']

pca = PCA(n_components=0.9)
environ_scale_data = data_scale_log[environ_feature]
pca.fit(environ_scale_data)
pca_environ = pd.DataFrame((pca.transform(environ_scale_data)))

num_of_principal = pca_environ.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 2


In [131]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=environ_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.6214892505570294
제 2주성분 : 0.29684237358975796


,0,1
미세먼지지수,0.676542,-0.145958
미세먼지백분위,0.644634,-0.356461
거주인구대비취약자수,-0.356003,-0.922839


In [132]:
environ_column = ['미세먼지 오염이 비교적 심한 곳', '비교적 공기오염이 덜한 곳']

### 상권 특성

In [133]:
market_feature = ['1점포당_1년_매출액', '승하차인원_버스_지하철', '종사자수 (명)', '사업체수 (개)']

pca = PCA(n_components=0.9)
market_scale_data = data_scale_log[market_feature]
pca.fit(market_scale_data)
pca_market = pd.DataFrame((pca.transform(market_scale_data)))

num_of_principal = pca_market.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 3


In [135]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=market_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.6305867737478807
제 2주성분 : 0.23598886514335926
제 3주성분 : 0.10662930670362157


,0,1,2
1점포당_1년_매출액,-0.194593,0.978359,0.036117
승하차인원_버스_지하철,-0.521853,-0.072574,-0.849939
종사자수 (명),-0.590866,-0.087148,0.368123
사업체수 (개),-0.583678,-0.173068,0.375214


In [136]:
market_column = ['사업체가 적은 곳', '상가 매출이 많은 곳', '사업체가 많은 곳']

## 차원축소 결과

In [138]:
pca_result =pd.concat([pca_population, pca_infra, pca_demand, pca_environ, pca_market],axis=1)

In [141]:
pca_result.columns= population_columns + infra_column + demand_column + environ_column + market_column
pca_result.index = data.index
pca_result

,인구가 비교적 적은 지역,1인 세대 및 20~30대 남녀가 많은 지역,"미취학 및 초등학생, 젊은 세대가 많은 지역",도시농업 체험 프로그램이 많은 지역,도시 농업 체험 프로그램이 적은 지역,예상 수요가 낮은 지역,유치원이 많은 지역,납품할 업체가 많은 지역,초등학교가 많은 지역,미세먼지 오염이 비교적 심한 곳,비교적 공기오염이 덜한 곳,사업체가 적은 곳,상가 매출이 많은 곳,사업체가 많은 곳
가락1동,-1.385345,-1.803981,0.717072,-0.524023,-0.220172,0.236120,0.070813,1.875716,1.158075,-0.094965,-0.641092,-0.856961,1.575414,0.946321
가락2동,-2.186509,-0.903525,0.560421,-0.235092,2.219405,-0.475947,1.330384,0.320708,-0.312788,-0.168193,-0.830915,-0.355864,-0.024727,0.723038
가락본동,-1.414738,0.052435,0.375364,-0.435469,0.527534,-2.229353,0.582526,0.817341,0.508024,0.005528,-0.380592,-1.581389,-0.395351,0.845590
가리봉동,5.988013,2.422976,-0.892778,-0.603970,-0.895200,1.654339,-1.617270,1.205942,1.438117,0.086447,0.640829,1.288125,-1.511582,-0.513801
가산동,-0.789125,2.595896,1.082431,-0.591722,-0.791785,-3.424138,-1.776846,0.331809,-0.905590,0.476478,0.258452,-5.306620,0.018415,0.860085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
효창동,2.345512,-0.479760,0.577316,-0.281735,1.825577,1.434872,-1.637105,-0.565325,1.236236,0.979602,0.857256,1.932959,1.239572,-0.249081
후암동,0.321338,0.281964,0.023078,-0.581655,-0.706781,0.513579,0.026451,-1.734819,0.757831,0.651105,0.005716,0.983336,0.248618,0.007313
휘경1동,1.008292,0.798911,0.616412,-0.580306,-0.695398,0.367425,-1.796529,-1.518520,-1.086658,-0.544548,1.181152,0.515704,0.184295,-1.189764
휘경2동,-1.530182,0.590519,0.310702,-0.569024,-0.600132,-0.284002,0.963473,-0.892087,-0.073695,-0.956301,0.113796,0.643531,-0.511455,-0.173268


In [142]:
pca_result.to_csv('./data_to_model/data_after_pca.csv')